In [ ]:
from sklearn.linear_model import Lasso
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv('sph6004_assignment1_data.csv')
#check the data shape
data.shape

In [ ]:
#drop the columns (the number of none are over 70%)
empty_percentage =data.isnull().sum()/len(data)* 100# Select the columns with less than 70% of empty values
data= data.loc[:,empty_percentage < 70]
data.shape

In [ ]:
#using knn to impute none
from sklearn.impute import KNNImputer
data_unfilled=data.iloc[:,5:]
imputer = KNNImputer(n_neighbors=5)
data_filled = imputer.fit_transform(data_unfilled)
data_filled = pd.DataFrame(data_filled, columns=data_unfilled.columns)
data_filled.head()

In [ ]:
data_filled=pd.concat([data.iloc[:,:5],data_filled],axis=1)
data_filled.columns, data_filled.index = data.columns, data.index

In [ ]:
data_filled.describe()

In [ ]:
#standardization_exclude age
scaler=preprocessing.StandardScaler()
data2scaler=pd.concat([data_filled.iloc[:,5:],data_filled[['admission_age']]],axis=1)
data_scaler=scaler.fit_transform(data2scaler)
data_scaler=pd.DataFrame(data_scaler)
data_scaler.columns, data_scaler.index = data2scaler.columns, data2scaler.index
#one hot encoding(race)
encode = preprocessing.OneHotEncoder( categories='auto', handle_unknown='ignore' )
data_encode=data_filled[['race']]
data_encode=encode.fit_transform(data_encode).toarray()
data_encode=pd.DataFrame(data_encode)
#gender
data_filled.loc[data_filled["gender"] == "M", "gender"] = 0
data_filled.loc[data_filled["gender"] == "F", "gender"] = 1
data_gender=data_filled.loc[:,'gender']

In [ ]:
#putting them together
data_prcsd=pd.concat([data_scaler,data_encode,data_gender],axis=1)
data_prcsd['gender']=pd.to_numeric(data_prcsd['gender']).astype('int64')

In [ ]:
data_x=data_prcsd
data_y=(data['aki'] != 0).astype(int)
dataset=pd.concat([data_x,data_y],axis=1)
dataset.rename(columns={"aki": "label"}
dataset.to_csv('processed_data.csv')

In [ ]:
#lr with l1 penlty
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pandas as pd
dataset=pd.read_csv('processed_data.csv')
data_x=sampled_df.iloc[:,:-1]
data_y=sampled_df.iloc[:,-1]
estimator = LogisticRegression(penalty='l1', solver='saga')
selector = RFE(estimator, n_features_to_select=50, step=1)
selector = selector.fit(data_x, data_y)
features_selected = selector.get_support(indices=True)
selected_features_df = pd.DataFrame(selector.transform(data_x), columns=data_x.columns[features_selected])
selected_features_df.to_csv('selected_l1.csv')

In [ ]:
#feature selection: stepwise regression
import pandas as pd
import toad
dataset=pd.read_csv('dataset.csv')
dataset.columns = dataset.columns.astype(str)
data_selected_stepwise=toad.selection.stepwise(dataset, target='label', estimator='ols', direction='both', criterion='aic')
data_selected_stepwise=pd.DataFrame(data_selected_stepwise)
data_selected_stepwise.to_csv('selected_stepwise.csv')

In [ ]:
from genetic_selection import GeneticSelectionCV
from sklearn.ensemble import RandomForestClassifier

dataset=pd.read_csv('dataset.csv')
dataset.columns = dataset.columns.astype(str)
data_x=dataset.iloc[:,:-1]
data_y=dataset.iloc[:,-1]
selector = GeneticSelectionCV(RandomForestClassifier(n_estimators=100),
                              cv=5,
                              verbose=0,
                              scoring="accuracy",
                              max_features=50,
                              n_population=80)
genetics = selector.fit(data_x, data_y)
features_selected = selector.get_support(indices=True)
data_selected_genetic = genetics.transform(data_x)
data_selected_genetic = pd.DataFrame(data_selected_genetic, columns=data_x.columns[features_selected])
data_selected_genetic.to_csv('selected_genetic.csv')